# 📦 Mercado Libre Data Engineer Challenge
### Este notebook genera datos simulados.

Instalación de paquetes

In [33]:
!pip install Faker

Importaciones y configuraciones globales

In [34]:
from faker import Faker
import pandas as pd
import random
import unicodedata
from datetime import datetime, timedelta
#Se obtiene ejemplos de Argentina
fake = Faker('es_AR')

Fuciones útiles

In [35]:
# Generar mail de acuerdo al nombre
def generar_email(nombre, apellido):
    dominio = random.choice(lista_dominios)
    return f"{limpiar(nombre)}.{limpiar(apellido)}{dominio}"

# Normalizar y limpar el mail
def limpiar(texto):
    return unicodedata.normalize('NFKD', texto.split()[0]).encode('ascii', 'ignore').decode('utf-8').lower()

Generar Clientes

In [36]:
# Dominios de mail
lista_dominios = ["@hotmail.com", "@gmail.com", "@outlook.com", "@yahoo.com", "@icloud.com"]

def generar_customers(n):
    customers = []
    for i in range(1, n + 1):
      nombre = fake.first_name()
      apellido = fake.last_name()
      customers.append({
          "customer_id": i,
          "nombre": nombre,
          "apellido": apellido,
          "email": generar_email(nombre, apellido),
          "sexo": random.choice(["M", "F"]),
          "direccion": fake.address().replace("\n", ", "),
          "nacimiento": fake.date_of_birth(minimum_age=18, maximum_age=85),
          "telefono": fake.phone_number()
      })
    return customers

Generar Categorías

In [37]:
# Genera base aleatoria de Categories
def generar_categories():
    categories = [
        {"category_id": 1, "nombre_categoria": "Celulares", "path_item": "Tecnología > Celulares y Teléfonos > Celulares y Smartphones"},
        {"category_id": 2, "nombre_categoria": "Notebooks", "path_item": "Tecnología > Computadoras > Notebooks"},
        {"category_id": 3, "nombre_categoria": "Cafeteras", "path_item": "Hogar > Electrodomésticos y Aires Ac. > Pequeños Electrodomésticos> Para Cocina > Cafeteras"},
    ]
    return categories

Generar Items

In [150]:
# Genera base aleatoria de Items
def generar_items(q=15, lista_clientes=None, por=5):
    if not lista_clientes or not isinstance(lista_clientes, list):
        raise ValueError("Debés pasar una lista válida de clientes.")

    # Lista que guarda los items
    items = []

    # Productos Principales por Categoria
    lista_celulares = ["Samsung Galaxy A15", "Xiaomi Redmi Note 13 Pro", "Motorola Moto G24 Power", "iPhone 16 Pro", "iPhone 17 Pro Max"]
    lista_notebooks = ["MacBook Pro 13", "HP 255 G10", "Lenovo Ideapad Slim 3", "Lenovo Ideapad Slim 3", "Samsung Galaxy Np750 Book3"]
    lista_cafeteras = ["Dolce Gusto Piccolo XS", " Smartchef Pe-ce5002 Automática Expreso P1", "Dolce Gusto Genio S", "Nespresso Essenza Mini D", "Peabody Smartchef Pe-ct4207"]

    # Obtener todos los customer_id desde la lista
    all_customer_ids = [c['customer_id'] for c in lista_clientes]

    # Determinar el %(parámetro por) de los clientes como vendedores
    total_vendedores = max(1, int(len(all_customer_ids) * por //100 ))
    seller_ids = random.sample(all_customer_ids, total_vendedores)

    # Mapear category_id a sus respectivas listas
    productos_por_categoria = {
        1: lista_celulares,
        2: lista_notebooks,
        3: lista_cafeteras
    }

    for i in range(1, q + 1):
        categoria_id = random.choice([1, 2, 3])
        titulo = random.choice(productos_por_categoria[categoria_id])

        items.append({
            "item_id": i,
            "titulo": titulo,
            "precio": random.randint(100, 1000),
            "status": random.choice(["active", "inactive"]),
            "category_id": categoria_id,
            "seller_id": random.choice(seller_ids)
        })

    return items

Generar Orders

In [117]:
# Genera base aleatoria de Orders
def generar_orders(o=4000, lista_items=None, lista_clientes=None):
    if not lista_items or not lista_clientes:
        raise ValueError("Debés pasar listas válidas de ítems y clientes.")

    orders = []
    fecha_base = datetime(2020, 1, 1)

    for i in range(1, o + 1):
        cantidad = random.randint(1, 4)
        order_date = fecha_base + timedelta(days=random.randint(0, 364))

        item = random.choice(lista_items)
        buyer = random.choice(lista_clientes)

        # Evitar que buyer y seller sean la misma persona (opcional)
        while buyer["customer_id"] == item["seller_id"]:
            buyer = random.choice(lista_clientes)

        orders.append({
            "order_id": i,
            "item_id": item["item_id"],
            "seller_id": item["seller_id"],
            "buyer_id": buyer["customer_id"],
            "cantidad": cantidad,
            "precio": item["precio"],
            "order_date": order_date.strftime('%Y-%m-%d')
        })

    return orders

In [151]:
n = int(input("Ingrese la cantidad de usuarios que va a tener la base: "))

Ingrese la cantidad de usuarios que va a tener la base: 10000


In [172]:
customers = generar_customers(n)

In [191]:
items = generar_items(q=1500, lista_clientes=customers,por=5)

In [108]:
categories = generar_categories()

In [192]:
orders = generar_orders(o=13000, lista_clientes=customers, lista_items=items)

Se crean Dataframes a partir de las Listas de entidades para explorar la información generada

In [193]:
df_customers = pd.DataFrame(customers)
df_items = pd.DataFrame(items)
df_categories = pd.DataFrame(categories)
df_orders = pd.DataFrame(orders)

In [46]:
df_customers.sample(10)

,customer_id,nombre,apellido,email,sexo,direccion,nacimiento,telefono
34,35,Tiziano,Hernandez,tiziano.hernandez@icloud.com,M,"Diagonal Bahía Blanca N° 879, Comodoro Rivadav...",1978-11-02,+54 15 2396 2987
330,331,Tomas,Flores,tomas.flores@icloud.com,F,"Av. Río Gallegos N° 6960 Oficina 4, La Plata 1...",1983-12-06,+54 15 2785 9091
332,333,Emilia,Gutierrez,emilia.gutierrez@gmail.com,M,"Avenida Mar del Plata N° 126 Torre 4 Dto. 4, C...",2004-11-07,+54 15 2405 4969
73,74,Jazmín,Maldonado,jazmin.maldonado@outlook.com,F,"Calle 8 N° 25, San Juan 5400, San Juan",1965-11-07,+54 9 3979 4639
459,460,Bautista,Vega,bautista.vega@outlook.com,F,"Avenida Viedma N° 748, Córdoba 5000, Córdoba",1985-11-12,+54 15 2987 5701
137,138,Dylan Benjamin,Leiva,dylan.leiva@gmail.com,M,"Av. Saavedra N° 2310 Dto. 5, San Miguel de Tuc...",1961-01-06,+54 9 3232 6109
176,177,Justina,Campos,justina.campos@gmail.com,M,"Blv. Santa Fe N° 3376 Piso 3 Dto. 9, Corriente...",1973-02-02,+54 15 2511 5885
454,455,Valentina,Acosta,valentina.acosta@yahoo.com,F,"Avenida Entre Ríos N° 871, Ushuaia 9410, Tierr...",1951-01-10,+54 9 3215 3419
440,441,Isabella,Sanchez,isabella.sanchez@yahoo.com,M,"Av. Belgrano N° 4085 Oficina 8, Rosario 2000, ...",1965-03-29,+54 15 2605 9199
101,102,Emiliano,Ramirez,emiliano.ramirez@hotmail.com,M,"Calle Salta N° 648 Torre 1 Dto. 3, Rosario 200...",2005-10-29,+54 9 3355 5454


In [47]:
df_items

,item_id,titulo,precio,status,category_id,seller_id
0,1,HP 255 G10,100681,active,2,245
1,2,HP 255 G10,52848,inactive,2,128
2,3,Dolce Gusto Genio S,173298,inactive,3,420
3,4,Lenovo Ideapad Slim 3,155485,active,2,135
4,5,Lenovo Ideapad Slim 3,227548,active,2,452
5,6,iPhone 17 Pro Max,203475,inactive,1,135
6,7,iPhone 17 Pro Max,249309,active,1,87
7,8,Lenovo Ideapad Slim 3,115684,active,2,277
8,9,Smartchef Pe-ce5002 Automática Expreso P1,110893,inactive,3,420
9,10,Dolce Gusto Genio S,200487,active,3,135


In [19]:
df_categories

,category_id,nombre_categoria,path
0,1,Celulares,Tecnología > Celulares y Teléfonos > Celulares...
1,2,Notebooks,Tecnología > Computadoras > Notebooks
2,3,Cafeteras,Hogar > Electrodomésticos y Aires Ac. > Pequeñ...


In [142]:
df_orders

,order_id,item_id,seller_id,buyer_id,cantidad,precio,order_date
0,1,161,2618,3257,4,66751,2020-06-03
1,2,235,979,4149,2,35293,2020-12-21
2,3,128,4475,1828,1,20952,2020-06-02
3,4,90,499,2577,3,25642,2020-01-01
4,5,201,3353,3289,2,206491,2020-07-05
...,...,...,...,...,...,...,...
12995,12996,131,2564,4571,4,166751,2020-01-26
12996,12997,180,2263,161,1,220806,2020-01-31
12997,12998,33,2275,3581,4,179450,2020-04-18
12998,12999,61,1894,2715,1,61275,2020-02-12


In [22]:
df_orders.sort_values(by='order_date', ascending=False)

,order_id,item_id,seller_id,buyer_id,cantidad,precio,order_date
3245,3246,6,31,6,4,224244,2020-12-30
1810,1811,1,45,32,1,92651,2020-12-30
1284,1285,11,31,24,1,104590,2020-12-30
254,255,11,31,10,1,104590,2020-12-30
476,477,11,31,32,4,104590,2020-12-30
...,...,...,...,...,...,...,...
246,247,8,45,22,2,10291,2020-01-01
1238,1239,6,31,50,2,224244,2020-01-01
96,97,13,45,38,1,20168,2020-01-01
2999,3000,10,45,36,4,124883,2020-01-01


Generar base para tener una tabla de Items Histórico por día

In [196]:
# Generar los últimos 5 días del 2020
start_date = datetime(2020, 12, 27)
end_date = datetime(2020, 12, 31)
delta = end_date - start_date

# Inicializar lista para snapshots
snapshots = []

for day in range(delta.days + 1):  # incluye el 31/12
    snapshot_date = (start_date + timedelta(days=day)).strftime("%Y-%m-%d")

    for _, row in df_items.iterrows():
        snapshots.append({
            "item_id": row["item_id"],
            "snapshot_date": snapshot_date,
            "precio": int(row["precio"] * random.uniform(0.95, 1.05)),
            "status": random.choice(["active", "inactive"])
        })

# Crear el DataFrame y exportarlo
df_hist = pd.DataFrame(snapshots)

In [197]:
df_hist

,item_id,snapshot_date,precio,status
0,1,2020-12-27,423,inactive
1,2,2020-12-27,652,inactive
2,3,2020-12-27,484,active
3,4,2020-12-27,904,active
4,5,2020-12-27,244,inactive
...,...,...,...,...
7495,1496,2020-12-31,291,inactive
7496,1497,2020-12-31,287,inactive
7497,1498,2020-12-31,833,active
7498,1499,2020-12-31,425,active


Guardo las bases en formato csv

In [198]:
pd.DataFrame(customers).to_csv("customers.csv", index=False)
pd.DataFrame(items).to_csv("items.csv", index=False)
pd.DataFrame(categories).to_csv("categories.csv", index=False)
pd.DataFrame(orders).to_csv("orders.csv", index=False)
pd.DataFrame(df_hist).to_csv("item_history.csv", index=False)